In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import pandas as pd
import owlready2
from elasticsearch import Elasticsearch
from mapping import *
from path import *
from konlpy.tag import Okt

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = owlready2.get_ontology(data_path).load()

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)

# Load unit path data
unit_path = pd.read_csv('../unit_path.csv')
G = generate_graph(unit_path)

In [4]:
# okt = Okt()

# query = input("Question: ") # 용감한 형제가 작곡한 노래의 장르는?
# query_pos = okt.pos(query, norm=True)

# query_terms = [q[0] for q in query_pos if q[1] not in ["Josa", "Punctuation"]]

In [5]:
query_terms =['용감한', '형제', '작곡', '노래', '장르']

In [6]:
query_patitions = query_patitioning(query_terms)
query_patitions

[['용감한 형제 작곡 노래 장르'],
 ['용감한', '형제 작곡 노래 장르'],
 ['용감한 형제', '작곡 노래 장르'],
 ['용감한 형제 작곡', '노래 장르'],
 ['용감한 형제 작곡 노래', '장르'],
 ['용감한', '형제', '작곡 노래 장르'],
 ['용감한', '형제 작곡', '노래 장르'],
 ['용감한', '형제 작곡 노래', '장르'],
 ['용감한 형제', '작곡', '노래 장르'],
 ['용감한 형제', '작곡 노래', '장르'],
 ['용감한 형제 작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래 장르'],
 ['용감한', '형제', '작곡 노래', '장르'],
 ['용감한', '형제 작곡', '노래', '장르'],
 ['용감한 형제', '작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래', '장르']]

In [7]:
p_star, score_list = partition_scoring(es, index_name, query_patitions)
p_star

['용감한 형제', '작곡', '노래', '장르']

In [8]:
candidates_list = [onto.search(label=query) for query in p_star]
resource_combinations = list(itertools.product(*candidates_list))
resource_combinations

[(SKMO_v2.4_0223.용감한_형제,
  SKMO_v2.4_0223.isComposedBy,
  mo.Track,
  SKMO_v2.4_0223.hasGenre),
 (SKMO_v2.4_0223.용감한_형제,
  SKMO_v2.4_0223.isComposedBy,
  mo.Track,
  SKMO_v2.4_0223.Genre)]

In [9]:
conceptual_graph = []

for rc in resource_combinations:
    conceptual_arc = []
    state_list = []
    rc = list(rc)

    for r in rc:
        if r in onto.object_properties():
            state_list.append('edge') 
        elif r in onto.data_properties():
            state_list.append('edge') 
        else:
            state_list.append('vertex') 

    # 마지막이 edge일 때
    if state_list[-1] == 'edge':
        state_list.append('vertex')
        rc.append(owlready2.Thing)

    # vertex - vertex 일때
    index_list = []
    for i in range(len(state_list)-1):
        if state_list[i] == 'vertex' and state_list[i+1] == 'vertex':
            index_list.append(i+1)
    
    for i in index_list:
        state_list.insert(i, 'edge')
        rc.insert(i, 'Any P')
    
    for offset in range(0,len(rc)-2, 2):
        conceptual_arc.append(tuple(rc[offset:offset+3]))
    
    print(conceptual_arc)

    conceptual_graph.append(conceptual_arc)

[(SKMO_v2.4_0223.용감한_형제, SKMO_v2.4_0223.isComposedBy, mo.Track), (mo.Track, SKMO_v2.4_0223.hasGenre, owl.Thing)]
[(SKMO_v2.4_0223.용감한_형제, SKMO_v2.4_0223.isComposedBy, mo.Track), (mo.Track, 'Any P', SKMO_v2.4_0223.Genre)]


In [10]:
candidates_dict = {}

for cg in conceptual_graph:
    
    for ca in cg:
        if ca in candidates_dict: 
            continue
        else:
            candidates_dict[ca] = []

        d,p,r = ca

        if d and not G.has_node(str(d)):
            d = d.is_a
        else: d = [d]

        if r and not G.has_node(str(r)):
            r = r.is_a
        else: r = [r]

        p = [p]

        for u,e,v in itertools.product(d,p,r):
            u = str(u)
            e = str(e)
            v = str(v)

            if e == 'Any P': e=None 
            if v == 'owl.Thing': v=None

            result = find_shortest_path(G, u, v, e, n_triple=3) 
            if len(result) == 0: continue
            candidates_dict[ca].append(result[::2])

In [12]:
for k,v in candidates_dict.items():
    print(f'{k}의 shortest path')
    for elem in v:
        print(elem)
    print()

(SKMO_v2.4_0223.용감한_형제, SKMO_v2.4_0223.isComposedBy, mo.Track)의 shortest path

(mo.Track, SKMO_v2.4_0223.hasGenre, owl.Thing)의 shortest path
['mo.Track', 'SKMO_v2.4_0223.Genre']

(mo.Track, 'Any P', SKMO_v2.4_0223.Genre)의 shortest path
['mo.Track', 'SKMO_v2.4_0223.Genre']

